# Concordance Analysis: Positive Sestamibi Scans

### Objective

Assess **localization concordance with surgical findings** for patients with **positive Sestamibi scans**.

### Data Source

- `side_concordance_results.xlsx` (pre-operative imaging and surgical outcomes)

### Outputs

- **Total scans with localization:** 113
- **Scans matching surgical side or exact gland:** 112 (**99.1%**)

---

In [1]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/side_concordance_results.xlsx')

In [2]:
def extract_side(location):
    if pd.isnull(location):
        return 'No Scan'
    
    if 'Left' in location:
        return 'Left'
    elif 'Right' in location:
        return 'Right'
    else:
        return 'Unknown'
    
preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)

In [3]:
def check_concordance(row, modality_col):
    imaging = row[modality_col]
    surgical = row['Surgical_Findings']
    
    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'
    
    imaging_glands = [g.strip() for g in imaging.split(',')]
    surgical_glands = [g.strip() for g in surgical.split(',')]
    
    # Exact gland match
    if any(sg in imaging_glands for sg in surgical_glands):
        return 'Yes (Exact)'
    
    # Side match
    imaging_sides = []
    for ig in imaging_glands:
        if 'Left' in ig:
            imaging_sides.append('Left')
        if 'Right' in ig:
            imaging_sides.append('Right')
    
    surgical_sides = []
    for sg in surgical_glands:
        if 'Left' in sg:
            surgical_sides.append('Left')
        if 'Right' in sg:
            surgical_sides.append('Right')
    
    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'
    
    return 'Incorrect Localization'

In [4]:
# Define modality
modality = 'Sestamibi'
imaging_side_col = 'Sestamibi Imaging Side'

# Apply extract_side function to get imaging side
preop_imaging_df[imaging_side_col] = preop_imaging_df[modality].apply(extract_side)

# Filter to scans that localized (Left or Right)
localized_sestamibi = preop_imaging_df[preop_imaging_df[imaging_side_col].isin(['Left', 'Right'])].copy()
total_localized_sestamibi = localized_sestamibi.shape[0]

# Extract surgical side if not already done
if 'Surgical Side' not in preop_imaging_df.columns:
    preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)

# Apply generalized check_concordance function
localized_sestamibi['Calculated Concordance'] = localized_sestamibi.apply(
    lambda row: check_concordance(row, modality), axis=1
)

In [5]:
# Count total concordant (either exact or side match)
total_concordant = localized_sestamibi['Calculated Concordance'].isin(['Yes (Exact)', 'Yes (Side)']).sum()

# Calculate percentage
side_concordance_percentage_sestamibi = (total_concordant / total_localized_sestamibi * 100) if total_localized_sestamibi > 0 else None

# Create a results dataframe for clean display
side_concordance_sestamibi_df = pd.DataFrame({
    'Metric': [
        'Total Sestamibi scans with localization',
        'Scans matching surgical side or exact gland'
    ],
    'Value': [
        total_localized_sestamibi,
        f"{total_concordant} ({side_concordance_percentage_sestamibi:.2f}%)"
    ]
})

# Display the dataframe
display(side_concordance_sestamibi_df)

,Metric,Value
0,Total Sestamibi scans with localization,113
1,Scans matching surgical side or exact gland,112 (99.12%)


---

### Interpretation

Positive Sestamibi scans demonstrated **high concordance (99%) with surgical findings**, indicating strong standalone diagnostic accuracy.



### Next Steps

- Compare to **SPECT/CT, 4D CT, and combined modality performance** to inform imaging pathway optimization in the manuscript.